In [95]:
from typing import TypedDict,Annotated,Optional,List
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,END,add_messages
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4
import json 
from langchain_core.messages import SystemMessage,HumanMessage,ToolMessage
from langgraph.prebuilt import ToolNode

In [96]:
load_dotenv()
memory = MemorySaver()

In [97]:
search_tool = TavilySearchResults(max_results=1)
tools = [search_tool]
llm = ChatGroq(model="llama-3.1-8b-instant")
llm_with_tools = llm.bind_tools(tools=tools)


In [98]:
class State(TypedDict):
    messages:Annotated[list,add_messages]

In [99]:
# model Node
async def model(state:State):
    llm_with_tools.ainvoke(state['messages'])

# Tool router
async def tool_router(state:State):
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and len(last_msg.tool_calls) > 0:
        return "tool_node"
    else:
        return END
    
# Tool Node
tool_node = ToolNode(tools=tools)

In [100]:
graph_builder = StateGraph(State)
graph_builder.add_node("model",model)
graph_builder.add_node("tool_node",tool_node)
graph_builder.set_entry_point("model")

In [101]:
graph_builder.add_edge("tool_node","model")
graph = graph_builder.compile(checkpointer=memory)

In [106]:
config = {
    "configurable":{
        "thread_id":5
    }
}

res = await graph.ainvoke({
    "messages":HumanMessage(content="How are you?")
},config=config)
res

C:\Users\aymaa\AppData\Local\Temp\ipykernel_32792\2411295684.py:3: RuntimeWarning: coroutine 'RunnableBindingBase.ainvoke' was never awaited
  llm_with_tools.ainvoke(state['messages'])


{'messages': [HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}, id='81f0d326-a9a9-48d0-8ba4-86726bd15686'),
  HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}, id='43030912-2cf8-4b5c-9253-f2fb46a4cab9'),
  HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}, id='0525af43-017d-4946-969f-c86bcc18c9bb'),
  HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}, id='10f6aab8-4fc4-426d-9983-7e86b4ef9ce9')]}